In [3]:
import gym
import numpy as np
import random

In [4]:
# class ShowerEnv(Env):
#     def __init__(self):
        
#         # Number of actions we can take (down, stay, up) (3)
#         self.action_space = Discrete(3)
        
#         # Temperature array
#         self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        
#         # Start temp
#         self.reset()
        
#     def step(self, action):
#         self.state += action - 1
#         self.shower_length -= 1
        
#         if self.state >= 37 and self.state <= 39:
#             reward = 1
#         else:
#             reward = -1
            
#         # Check if shower is done
#         if self.shower_length <= 0:
#             done = True
#         else:
#             done = False
            
#         # Apply temperature noise
#         self.state += random.randint(-1, 1)
#         # set placeholder for info
#         info = {}
        
#         return self.state, reward, done, info
    
#     def render(self):
#         # Implement visualisation
#         pass
    
#     def reset(self):
#         # Reset temp and shower length
#         self.state = 38 + random.randint(-3, 3)
#         self.shower_length = 60
#         return self.state

In [5]:
env_name = "LunarLander-v2"
env = gym.make(env_name)
# env = ShowerEnv()
states = env.observation_space.shape[0]
actions = env.action_space.n

In [6]:
actions

4

In [7]:
episodes = 10
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        
    print('Episode:', episode, "Score:", score)

Episode: 1 Score: -117.95880041615695
Episode: 2 Score: -103.91612801710234
Episode: 3 Score: -215.97948015420195
Episode: 4 Score: -136.6961229089929
Episode: 5 Score: -118.24100917490036
Episode: 6 Score: -99.94820646742372
Episode: 7 Score: -186.66751735895662
Episode: 8 Score: -87.70664476982633
Episode: 9 Score: -216.81804321666763
Episode: 10 Score: -76.9798249006105


# Create a deep learning model with Keras

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [9]:
states = env.observation_space.shape[0]
actions = env.action_space.n
print(states, actions)

8 4


In [27]:
def build_model(states, actions):
    model = tf.keras.Sequential()

    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [28]:
model = build_model(states, actions)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 8)                 0         
                                                                 
 dense_9 (Dense)             (None, 256)               2304      
                                                                 
 dense_10 (Dense)            (None, 256)               65792     
                                                                 
 dense_11 (Dense)            (None, 4)                 1028      
                                                                 
Total params: 69,124
Trainable params: 69,124
Non-trainable params: 0
_________________________________________________________________


# Build Agent with Keras-RL

In [29]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [30]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, 
                   memory=memory, 
                   policy=policy, 
                   nb_actions=actions, 
                   nb_steps_warmup=10, 
                   target_model_update=1e-2)
    
    return dqn

In [31]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 60s 6ms/step - reward: -0.3661
24 episodes - episode_reward: -154.832 [-567.273, -13.283] - loss: 6.956 - mae: 19.047 - mean_q: 10.435

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 62s 6ms/step - reward: -0.0212
16 episodes - episode_reward: -18.546 [-259.508, 90.728] - loss: 4.351 - mae: 28.469 - mean_q: 35.967

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 62s 6ms/step - reward: 0.0140
12 episodes - episode_reward: 16.224 [-166.915, 127.618] - loss: 4.707 - mae: 34.864 - mean_q: 45.499

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 64s 6ms/step - reward: 0.0636
11 episodes - episode_reward: 53.826 [-197.202, 133.078] - loss: 4.680 - mae: 34.678 - mean_q: 45.633

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 63s 6ms/step - reward: 0.0616

# Testing model

In [32]:
dqn.test(env, nb_episodes=25, visualize=True)

Testing for 25 episodes ...
Episode 1: reward: 141.242, steps: 1000
Episode 2: reward: -44.191, steps: 1000
Episode 3: reward: 276.863, steps: 243
Episode 4: reward: 293.085, steps: 364
Episode 5: reward: 226.107, steps: 217
Episode 6: reward: 18.962, steps: 214
Episode 7: reward: 220.087, steps: 944
Episode 8: reward: 274.424, steps: 211
Episode 9: reward: 144.208, steps: 1000
Episode 10: reward: 188.117, steps: 260
Episode 11: reward: 150.954, steps: 1000
Episode 12: reward: 268.522, steps: 223
Episode 13: reward: 300.517, steps: 217
Episode 14: reward: 139.672, steps: 1000
Episode 15: reward: 102.748, steps: 1000
Episode 16: reward: 191.559, steps: 355
Episode 17: reward: 167.681, steps: 1000
Episode 18: reward: 256.065, steps: 215
Episode 19: reward: 277.178, steps: 258
Episode 20: reward: 219.811, steps: 898
Episode 21: reward: 248.608, steps: 204
Episode 22: reward: -531.440, steps: 627
Episode 23: reward: 228.840, steps: 288
Episode 24: reward: 268.318, steps: 254
Episode 25: re

# Save and reload model

In [ ]:
dqn.save_weights('dqn_lander_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make(env_name)
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
dqn.test(env, nb_episodes=25, visualize=True)